# FFT in Multiple Dimensions

We can extend our FFT to more than one dimension.  Consider the 2-d case:

\begin{align*}
F_{k_x,k_y} &= \sum_{m =0}^{N_x - 1} \sum_{n=0}^{N_y -1}
f_{mn} e^{-2\pi i (k_x m/N_x + k_y n/N_y)} \\
 &= \sum_{m =0}^{N_x - 1} 
e^{-2\pi i k_x m/N_x}
\underbrace{\sum_{n=0}^{N_y -1} f_{mn} e^{-2\pi i k_y n/N_y}}_{\mbox{this is the y transform}}
\end{align*}

We see that we can decompose the multi-dimensional transform into a sequence of one-dimensional FFTs.

## Application: Turbulent Power Spectrum

One of the ways this is used frequently in astrophysics is to compute the power spectrum of a velocity field to look at the
[turbulence](https://en.wikipedia.org/wiki/Turbulence) properties.

For a simulation with velocity components $u$, $v$, and $w$, we compute the power spectrum as:

$$E(k) = \int_{k=|k|} dk [ \hat{u}(k)^2 + \hat{v}(k)^2 + \hat{w}(k)^2 ]$$

where $k$ is the radial wavenumber, $k = \sqrt{k_x^2 + k_y^2 + k_z^2}$.  This gives us the power at a scake $k$.

Kolmogorov turbulence theory says that homogeneous, isotropic, incompressible turblence should scale like:

$$E(k) dk \sim k^{-5/3}$$

We can see this behavior, for example here: https://ui.adsabs.harvard.edu/abs/2005ApJ...632.1021Z/abstract